In [55]:
from lbl2vec import Lbl2Vec
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import numpy as np
from utils import *

In [65]:
labels = ['planete', 'sport', 'economie', 'arts-stars', 'high-tech', 'politique', 'monde', 'societe', 'faits_divers', 'sante', 'justice']
climat_kw = ['climat', "environnement", "température", "réchauffement", "nature"]
sport_kw = ['sport', "football", "rugby", "match", "compétition", "score", "coupe", "course", "basket"]
economie_kw = ["économie", "argent", "inflation", "coût", "euro", "impôt", "dette", "croissance"]
technologie_kw = ["technologie", "ordinateur", 'téléphone', "serveur", "high-tech", "écran", "nanotechnologies"]
politique_kw = ["politique", "vote", "élection", "référendum", "parlement", "ministre", "débat"]
diverstissement_kw = ["divertissement", "cinéma", "stars", "théâtre"]
sante_kw = ["santé", "maladie", "méicament"]
justice_kw = ["justice", "condamnation", "juger"]
faits_divers_kw = ["fait", "divers", "fait divers"]
monde_kw = ["monde", "guerre", "pays"]
societe_kw = ["société", "manifestation", "population"]


descriptive_keywords = [climat_kw, sport_kw, economie_kw, diverstissement_kw, technologie_kw, politique_kw, monde_kw, societe_kw, faits_divers_kw, sante_kw, justice_kw]

In [ ]:
df_articles = load_newspaper()

In [66]:
train_dataset, test_dataset = extract_train_test_dataset(df_articles)

In [67]:
tagged_docs = []
for index, row in tqdm(train_dataset.iterrows()):
    text = row['body'].lower().split()
    tag = row['label']
    tagged_docs.append(TaggedDocument(text, [tag]))

17000it [00:01, 14920.32it/s]


In [68]:
# init model
model = Lbl2Vec(keywords_list=descriptive_keywords, tagged_documents=tagged_docs, label_names=labels)
# train model
model.fit()

2022-12-21 17:31:25,562 - Lbl2Vec - INFO - Train document and word embeddings
2022-12-21 17:34:17,893 - Lbl2Vec - INFO - Train label embeddings
2022-12-21 17:34:17,898 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: impôt
2022-12-21 17:34:17,900 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: astronomie Einstein
2022-12-21 17:34:17,902 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: high-tech nanotechnologies


In [69]:
# get similarity scores from trained model
model.predict_model_docs()

2022-12-21 17:34:26,565 - Lbl2Vec - INFO - Get document embeddings from model
2022-12-21 17:34:26,566 - Lbl2Vec - INFO - Calculate document<->label similarities


,doc_key,most_similar_label,highest_similarity_score,climat,sport,economie,sciences,technologie,politique
0,0,climat,1.0,1.000000,-0.038134,0.164720,0.296873,0.094864,-0.009796
1,1,sport,1.0,-0.038134,1.000000,0.040454,0.130160,0.068155,0.123125
2,2,economie,1.0,0.164720,0.040454,1.000000,-0.012358,0.208165,0.174219
3,3,sciences,1.0,0.296873,0.130160,-0.012358,1.000000,0.218228,0.026183
4,4,technologie,1.0,0.094864,0.068155,0.208165,0.218228,1.000000,-0.062242
5,5,politique,1.0,-0.009796,0.123125,0.174219,0.026183,-0.062242,1.000000


In [70]:
tagged_docs_test = []
y_true = []
for index, row in tqdm(test_dataset.iterrows()):
    text = row['body'].lower().split()
    tag = str(index)
    y_true.append(row['label'])
    tagged_docs_test.append(TaggedDocument(text, [tag]))

5500it [00:00, 19528.22it/s]


In [71]:
results = model.predict_new_docs(tagged_docs=tagged_docs_test)

2022-12-21 17:34:43,643 - Lbl2Vec - INFO - Calculate document embeddings
2022-12-21 17:34:55,664 - Lbl2Vec - INFO - Calculate document<->label similarities


In [72]:
dict_labels_model = {'planete': 0, 'sport': 1, 'economie': 2, 'arts-stars': 3, 'high-tech': 4, 'politique': 5, 'monde': 6, 'societe': 7, 'faits_divers': 8, 'sante': 9, 'justice': 10}
r = results.most_similar_label.to_list()
y_pred = []
for lab in r:
    y_pred.append(dict_labels_model[lab])

In [74]:
acc = np.sum(np.array(y_pred) == np.array(y_true)) / len(y_pred)
acc

0.7483636363636363

In [77]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(np.array(y_true), np.array(y_pred))
cm

array([[386,  82, 116, 199,  59, 158],
       [ 13, 932,  12,  10,  13,  20],
       [ 58,  56, 586,  22, 107, 171],
       [ 18,  15,   2, 436,  21,   8],
       [ 13,  30,  25,  54, 827,  51],
       [  2,  13,  28,   2,   6, 949]], dtype=int64)